In [11]:
import pandas as pd
import pychoco as ch

In [8]:
# df = pd.read_csv("test.csv")
df = pd.read_csv("activ_total.csv")

user_label = "User"
hrs_label = "Available Hours"

df

,User,Activity 1,Activity 2,Activity 3,Activity 4,Activity 5,Available Hours
0,A,9,0,13,36,2,10.75
1,B,9,3,13,49,1,2.50
2,C,17,1,8,19,0,7.50
3,D,21,3,10,44,1,7.50
4,E,16,3,5,37,0,7.50
5,F,13,0,9,9,0,7.50


In [12]:
df = pd.read_csv("activity_counts.csv")
user_label = ""
hrs_label = "Hrs to Complete"

df

,Biometric Roll-Up Failed,File Check,No Match,Ready For Review,Verification Completed No Adverse,Final Withdrawn,Verification Requested,Final Refused,Final Approved,Ready for Approval,Verification Completed Adverse,Medical Overdue,Hrs to Complete
0,1,0,0,0,0,0,0,0,0,0,0,0,7.5
1,1,1,0,0,0,0,0,0,0,0,0,0,7.5
2,0,1,0,0,0,0,0,0,0,0,0,0,7.5
3,0,1,0,0,0,0,0,0,0,0,0,0,7.5
4,0,1,0,0,0,0,0,0,0,0,0,0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,0,0,0,0,0,0,0,1,0,0,0,0,7.5
427,0,0,0,0,0,0,0,2,0,0,0,0,7.5
428,0,0,0,0,0,0,0,3,0,0,0,0,7.5
429,0,0,0,0,0,0,0,1,0,0,0,0,7.5


In [38]:
df['Final Refused'].value_counts().reset_index().sort_values(by='Final Refused')

,Final Refused,count
0,0,264
1,1,65
2,2,28
3,3,16
4,4,9
5,5,8
6,6,7
7,7,5
13,8,2
9,10,3


In [13]:
activ_labels = [ col for col in df.columns if col not in [ hrs_label, user_label ]  ]
activ_labels

['Biometric Roll-Up Failed',
 'File Check',
 'No Match',
 'Ready For Review',
 'Verification Completed No Adverse',
 'Final Withdrawn',
 'Verification Requested',
 'Final Refused',
 'Final Approved',
 'Ready for Approval',
 'Verification Completed Adverse',
 'Medical Overdue']

In [27]:
# https://github.com/chocoteam/pychoco/tree/master

slack = 360 # (min)

model = ch.Model("ircc")

activ_vars = { activ_label: model.intvar(1, 8 * 60, name=activ_label) for activ_label in activ_labels }

cnt = 0
for idx, row in df.iterrows():
    cnt += 1
    
    activs = row[activ_labels]
    activs = activs.astype('int64')
    total_time = int(row[hrs_label] * 60)
    
    sum_vars = []
    for activ_label, num in activs[activs > 0].items():
        activ_var = activ_vars[activ_label]
        op = model.int_scale_view(activ_var, num)
        print(f"{" + " if len(sum_vars) > 0 else ""} (<{activ_label}> x {num})")
        sum_vars.append(op)
    
    print(f" = {total_time}")
    sum1 = model.sum(sum_vars, ">=", total_time - slack)
    sum2 = model.sum(sum_vars, "<=", total_time + slack)
    sum1.post()
    sum2.post()

    # if cnt == 2:
    #     break

 (<Biometric Roll-Up Failed> x 1)
 = 450
 (<Biometric Roll-Up Failed> x 1)
 +  (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 8)
 = 450
 (<File Check> x 2)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 11)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 3)
 = 450
 (<Biometric Roll-Up Failed> x 2)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 3)
 = 450
 (<File Check> x 9)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 3)
 = 450
 (<File Check> x 1)
 = 450
 (<Biometric Roll-Up Failed> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<Biometric Roll-Up Failed> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<No Match> x 1)
 = 450
 (<File Check> x 1)
 = 450
 (<File Check> x 3)
 = 450
 (<File Check> x 3)
 = 450
 (<File Check> x 1)
 = 450
 (<No Match> x 1)
 = 450
 (<File Check> x 7)

In [28]:
activ_var_values = { activ_label: set() for activ_label in activ_vars.keys() }

cnt = 0
while model.get_solver().solve():
    cnt += 1
    
    sol = { activ_label: var.get_value() for activ_label, var in activ_vars.items() }
    print("\n".join([f"{activ_label} = {value}" for activ_label, value in sol.items()]))
    
    for activ_label, value in sol.items():
        activ_var_values[activ_label].add(value)
        
    # if cnt == 10:
    #     break

print()
print(activ_var_values)


{'Biometric Roll-Up Failed': set(), 'File Check': set(), 'No Match': set(), 'Ready For Review': set(), 'Verification Completed No Adverse': set(), 'Final Withdrawn': set(), 'Verification Requested': set(), 'Final Refused': set(), 'Final Approved': set(), 'Ready for Approval': set(), 'Verification Completed Adverse': set(), 'Medical Overdue': set()}
